<h1> 2d. Distributed training and monitoring </h1>

In this notebook, we refactor to call ```train_and_evaluate``` instead of hand-coding our ML pipeline. This allows us to carry out evaluation as part of our training loop instead of as a separate step. It also adds in failure-handling that is necessary for distributed training capabilities.

We also use TensorBoard to monitor the training.

In [ ]:
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

<h2> Input </h2>

Read data created in Lab1a, but this time make it more general, so that we are reading in batches.  Instead of using Pandas, we will use add a filename queue to the TensorFlow graph.

In [ ]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURE_COLUMNS = CSV_COLUMNS[1:len(CSV_COLUMNS)-1]
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512):
  def decode_csv(value_column):
    columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label

  # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
  filenames_dataset = tf.data.Dataset.list_files(filename)
  # Read lines from text files
  textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
  # Parse text lines as comma-separated values (CSV)
  dataset = textlines_dataset.map(decode_csv)

  # Note:
  # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
  # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

  if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # indefinitely
      dataset = dataset.shuffle(buffer_size = 10 * batch_size)
  else:
      num_epochs = 1 # end-of-input after this

  dataset = dataset.repeat(num_epochs).batch(batch_size)

  return dataset

def get_train_input_fn():
  return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid_input_fn():
  return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> Create features out of input data </h2>

For now, pass these through.  (same as previous lab)

In [ ]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Serving input function </h2>
Defines the expected shape of the JSON feed that the modelwill receive once deployed behind a REST API in production.

In [ ]:
def serving_input_fn():
  json_feature_placeholders = dict(
      [ [feature, tf.placeholder(tf.float32, [None])] for feature in FEATURE_COLUMNS ]
  )
  features = dict( [feature, json_feature_placeholders[feature]] for feature in FEATURE_COLUMNS )
  return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

serving_input_fn()

<h2> tf.estimator.train_and_evaluate </h2>

In [ ]:
def train_and_evaluate(output_dir, num_train_steps):
  run_config = tf.estimator.RunConfig(
      model_dir=output_dir,
      save_summary_steps=100,
      save_checkpoints_steps=num_train_steps
  )
  
  estimator = tf.estimator.DNNRegressor(hidden_units=[32, 8, 2], feature_columns=feature_cols, config=run_config)
  
  train_spec = tf.estimator.TrainSpec(input_fn=get_train_input_fn)
  
  export_latest = tf.estimator.LatestExporter("exp", serving_input_receiver_fn=serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(input_fn=get_valid_input_fn, exporters=export_latest)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## Monitor training with TensorBoard

To activate TensorBoard within the JupyterLab UI navigate to "<b>File</b>" - "<b>New Launcher</b>".   Then double-click the 'Tensorboard' icon on the bottom row.

TensorBoard 1 will appear in the new tab.  Navigate through the three tabs to see the active TensorBoard.   The 'Graphs' and 'Projector' tabs offer very interesting information including the ability to replay the tests.

You may close the TensorBoard tab when you are finished exploring.

In [ ]:
OUTDIR = './taxi_trained'

<h2>Run training</h2>

In [ ]:
# Run training    
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
train_and_evaluate(OUTDIR, num_train_steps = 2000)

## Challenge Exercise

Modify your solution to the challenge exercise in c_dataset.ipynb appropriately.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

import random
import math

COLUMNS = ['key', 'volume', 'height', 'radius']

def get_random():
    random_range = random.random()*1.5+0.5
    return random_range

def get_data(count):
    data = []
    for i in range(count):
        random_h = get_random()
        random_r = get_random()
        volume = math.pow(random_r, 2) * math.pi * random_h

        # map features with 10% noise and round to nearest decimal place
        volume, random_h, random_r = map(
            lambda x: round(x + x*(random.random()*0.2-0.1), 1),
            [volume, random_h, random_r]
        )
        data.append([i, volume, random_h, random_r])
    numpy_data = np.array(data)
    df = pd.DataFrame(data=numpy_data[:,1:], index=numpy_data[:,0], columns=COLUMNS[1:])
    return df

df_train_cylinder = get_data(100000)
df_valid_cylinder = get_data(100000)

# now let's do the network stuff
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df['volume'],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df['volume'],
    batch_size = 128,
    num_epochs = 10,
    shuffle = True,
    queue_capacity = 1000
  )
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    batch_size = 128,
    num_epochs = 10,
    shuffle = True,
    queue_capacity = 1000
  )

def serving_input_fn_cylinder():
  json_feature_placeholders = {
      'height': tf.placeholder(tf.float32, [None]),
      'radius': tf.placeholder(tf.float32, [None])
  }
  features = dict( [feature, json_feature_placeholders[feature]] for feature in json_feature_placeholders.keys() )
  return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

featcols_cylinder = [
    tf.feature_column.numeric_column("height"),
    tf.feature_column.numeric_column("radius")
]

def train_and_evaluate_cylinder(output_dir, num_train_steps):
  run_config = tf.estimator.RunConfig(
      model_dir=output_dir,
      save_summary_steps=100,
      save_checkpoints_steps=num_train_steps
  )
  
  estimator = tf.estimator.DNNRegressor(
      hidden_units=[32, 8, 2],
      feature_columns=featcols_cylinder,
      config=run_config
  )
  
  train_spec = tf.estimator.TrainSpec(
      input_fn = make_train_input_fn(df=df_train_cylinder, num_epochs=10)
  )
  
  export_latest = tf.estimator.LatestExporter("exp", serving_input_receiver_fn=serving_input_fn_cylinder)
  eval_spec = tf.estimator.EvalSpec(input_fn=make_eval_input_fn(df_valid_cylinder), exporters=export_latest)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  return estimator

tf.logging.set_verbosity(tf.logging.WARN)

OUTDIR_CYLINDER = 'cylinder_data'

# Run training
shutil.rmtree(OUTDIR_CYLINDER, ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
model_cylinder_dnn= train_and_evaluate_cylinder(OUTDIR_CYLINDER, num_train_steps = 2000)

for i in range(5):
    # predict on random input row
    df_test_cylinder = get_data(1)
    print(df_test_cylinder)
    cylinder_predictions_dnn = model_cylinder_dnn.predict(make_prediction_input_fn(df_test_cylinder))
    print(next(cylinder_predictions_dnn))

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License